In [2]:
import torch 
import torch.nn as nn

import pprint 
pp = pprint.PrettyPrinter()


Tensors 

In [4]:
list_of_lists = [[1 ,2, 3], [4,5,6]]
print(list_of_lists)

[[1, 2, 3], [4, 5, 6]]


In [7]:
#init a tensor
data = torch.tensor([
    [0,1],
    [2,3],
    [4,5]
])
data

tensor([[0, 1],
        [2, 3],
        [4, 5]])

In [10]:
# we can cast them to different types -- usualloy float32 or int
data = torch.tensor([
    [0,1],
    [2,3],
    [4,5],
] ,   dtype=torch.float32)
data

tensor([[0., 1.],
        [2., 3.],
        [4., 5.]])

In [11]:
data.shape

torch.Size([3, 2])

In [24]:
data2 = torch.tensor([
    [1, 2, 3],
    [4, 5, 6]
], dtype=torch.float32)

# must be same type 
data3 = data @ data2

In [16]:
rr = torch.arange(1, 10)
rr

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [22]:
# can reshape a vector into a matrix!  
# evidently this can make batch operations easier 
# e.x. take a 15 element row vector and turn it into a 5 x 3 matrix 
vec = torch.arange(1,16)
print(vec)
matrix = vec.view(5,3)
print(matrix)
print(matrix.shape)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15]])
torch.Size([5, 3])


In [23]:
# one of the reason we use tensors is vectorized operations: 
# operations that be conducted in parallel over a paarticular dimension of a tensor 


In [28]:
# things we can do like sum, mean, and std dev 

# https://stackoverflow.com/questions/55691819/why-does-dim-1-return-row-indices-in-torch-argmax

# dim=1 is ROWS 
data3.mean(dim=1)
data3.sum(dim=1)

# dim=0 is COLUMNS
data3.mean(dim=0)
data3.sum(dim=0)

# look at the official pytorch tutorial for a good example of all the way you can slice a tensor


tensor([42., 57., 72.])

Autograd 

Pytorch is known for it's auto differentation feature.. We can call the `backward()` method to ask Pytorch to calculate the gradients which are then stored in the `grad` attribute

In [33]:
x = torch.tensor([2], dtype=torch.float32, requires_grad=True)

pp.pprint(x.grad)

None


In [53]:
y = 3 * x * x 
# z = 3 * x * x
print(type(y))
y.backward()
# if we just want to manually reset the grad instead of using an optimizer 
# we can just do it manually like this... 
# x.grad = torch.tensor([0.])
# x.zero_grad()
x.grad

<class 'torch.Tensor'>


tensor([84.])

# Neural Network Module

### Linear Layer 

we can use `nn.Linear(H_in, H_out)` to create a linear layer. This will take a matrix of (N, *, H_in) dimensions and ouptput a matrix of (N, *, H_out). the `*` denotes that there could be arbitrary number of dimensions in between. The linear layer performs the operation `Ax+b`., whewr e`A` and `b` are intialized randomly. If we don't wnat the linear layer to learn the bias parameters, we can intialize our layer with `bias=False`


In [88]:
import torch.nn as nn

# create the inputs 
# two, 3x4 matrices of ones. 
# 3 rows by 4 columns 
inputs = torch.ones(2,3,4)
# print(inputs)

# make a linear layers transforming N,*,H_in dimensional inputs to N,*,H_out dimensional outputs 
linear = nn.Linear(4,2)
linear_output = linear(inputs)
linear_output.shape
linear_output

tensor([[[ 0.1580, -0.3979],
         [ 0.1580, -0.3979],
         [ 0.1580, -0.3979]],

        [[ 0.1580, -0.3979],
         [ 0.1580, -0.3979],
         [ 0.1580, -0.3979]]], grad_fn=<AddBackward0>)

We get different values in the tensor each time we run it because `Linear()` randomly initializes the function `Ax + b` to different values each time!

In [90]:
list(linear.parameters())

[Parameter containing:
 tensor([[-0.2625, -0.1494,  0.0883,  0.1954],
         [ 0.0474,  0.1861,  0.1699, -0.3701]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2862, -0.4312], requires_grad=True)]

# Activation Function Layer

we can also use `nn` module to apply activations function to our tensors. Activation functions are used to add non-linearity to our network. Some examples of activations functions are `nn.ReLU()`, `nn.Sigmoid()` and `nn.LeakyReLU()`. Activation functions operate on each elementate seperatel, so sthe sahpe of the tensorwe get as an output ar ethe same as the ones we pass in.

In [92]:
sigmoid = nn.Sigmoid()

output = sigmoid(linear_output)
output


tensor([[[0.5394, 0.4018],
         [0.5394, 0.4018],
         [0.5394, 0.4018]],

        [[0.5394, 0.4018],
         [0.5394, 0.4018],
         [0.5394, 0.4018]]], grad_fn=<SigmoidBackward0>)

# Putting the Layers Together

So far we have just created lyaers and pass the output of one as the input of the next. Instead of creating intermediate tensors and passing them around, we can use `nn.Sequeuntial` which puts layers together sequentially

In [101]:
block = nn.Sequential(
    nn.Linear(4,2),
    nn.Linear(2,1),
    nn.Sigmoid(),
)

In [102]:
input = torch.ones(2,3,4)
output = block(input)
output

tensor([[[0.4056],
         [0.4056],
         [0.4056]],

        [[0.4056],
         [0.4056],
         [0.4056]]], grad_fn=<SigmoidBackward0>)

# Custom Modules 

Instead of using the predefined modules, we can also build our own by extending the `nn.Module` class. For example , we can build the `nn.Linear` on our own using te tensor introduced earlier! We can also build new, more complex modules, such a s a custom neural network. 

To create a custom module, the first thing we have to do is to extend the `nn.Module` . We can then initialize our parameters in the `__init__` function, staring with a call to the `__init__` function of the super class. Al the class attributes we define which are `nn` module objects are treated as parameters , which can then learned during the training. Tensors are not parameters , but they can be turned into parameters if they are wrapp in `nn.Parameters` class 

All classes extending the `nn.Module` are also expected to implement a `forward(x)` function, where `x` is the tensor. this is the function that is called when a parameters is passed to our modul, for example `modelFooBarBaz(x)`

In [107]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, hidden_size):
        # Call to the super() class 
        super(MultiLayerPerceptron, self).__init__()

        # saving initialization params
        self.input_size = input_size 
        self.hidden_size = hidden_size

        #define our model 
        # doesn't have to be named model 
        self.model = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, self.input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        return output

        



In [108]:
# now lets instantiate it and see what it does. 

input = torch.randn(2,5)

multi_layer_perceptron = MultiLayerPerceptron(5,3)

#pass our input through our model 
multi_layer_perceptron(input)


tensor([[0.3950, 0.4821, 0.3842, 0.5562, 0.4945],
        [0.4746, 0.5936, 0.3286, 0.4435, 0.3822]], grad_fn=<SigmoidBackward0>)

In [109]:
# we can inspect the parameters of our model with `named_parameters()` and `parameters()`
list(multi_layer_perceptron.named_parameters())

[('model.0.weight',
  Parameter containing:
  tensor([[-0.3665, -0.0611,  0.4206, -0.1927, -0.2036],
          [-0.3317, -0.4113, -0.1653, -0.2854, -0.1773],
          [-0.4202,  0.1413,  0.0547, -0.0287, -0.4163]], requires_grad=True)),
 ('model.0.bias',
  Parameter containing:
  tensor([ 0.4075, -0.2855, -0.0500], requires_grad=True)),
 ('model.2.weight',
  Parameter containing:
  tensor([[ 0.1899,  0.2779,  0.1600],
          [ 0.1802,  0.2193,  0.5014],
          [-0.2587,  0.1474,  0.2716],
          [-0.3371,  0.4262,  0.0181],
          [-0.4094,  0.2103,  0.2476]], requires_grad=True)),
 ('model.2.bias',
  Parameter containing:
  tensor([-0.4264, -0.0715, -0.4720,  0.2259, -0.0221], requires_grad=True))]

In [116]:
list(multi_layer_perceptron.parameters())

[Parameter containing:
 tensor([[-0.1681,  0.3615, -0.1085,  0.2136, -0.3068],
         [ 0.1105, -0.0750, -0.4361,  0.4284,  0.3173],
         [ 0.1914,  0.1469,  0.4325, -0.1647, -0.3230]], requires_grad=True),
 Parameter containing:
 tensor([-0.0533, -0.1809, -0.3992], requires_grad=True),
 Parameter containing:
 tensor([[ 0.5454,  0.0090, -0.0701],
         [ 0.4629, -0.2518, -0.4816],
         [ 0.0641,  0.3194,  0.4202],
         [ 0.5256,  0.1806, -0.4929],
         [ 0.0383,  0.4789, -0.3110]], requires_grad=True),
 Parameter containing:
 tensor([-0.0959,  0.4256, -0.3175, -0.5408, -0.3457], requires_grad=True)]

# Optimization 

We have showed how gradients are calculated with the `backward()` function. Having the gradients isn't enough for our models to learn. We also need to know how to update the parameters of our models. this is where the optimizers comes in. `torch.optim` module contains several optimizers that we can use. Some popular examples are `optim.SGD` and `optim.Adam`. When initalizing optimizers, we pass our model parameters which can be accessed with `model.parameters()`, tellin g the optimizers which values it will optimizing. Optimizers also have a learning rate `lr` parameters, which determines how big of a n update will be made in every step. Different optimizers have different hypterparameters as well. 

after we have our optimization function, we can defien a loss that we want to optimize for. We can either define the loss ourselve, or use one of th epredefined loss function in `pyTorch`, such as `nn.BCELoss()`. Let's put everything together now! We will start by creating some dummy data

In [112]:
import torch.optim as optim 

# create label/y/output data 
y = torch.ones(10, 5)

# add some noise to our y to geenrate our x 
x = y + torch.rand_like(y)
x


tensor([[1.6229, 1.6178, 1.8535, 1.6113, 1.0269],
        [1.2863, 1.9466, 1.5752, 1.0483, 1.0441],
        [1.9156, 1.2305, 1.4923, 1.9213, 1.9353],
        [1.1242, 1.0729, 1.3030, 1.0789, 1.0008],
        [1.9523, 1.0653, 1.9383, 1.7856, 1.3598],
        [1.2001, 1.4937, 1.9633, 1.7623, 1.8388],
        [1.9458, 1.6999, 1.5424, 1.1136, 1.2931],
        [1.3253, 1.6970, 1.1702, 1.5542, 1.9958],
        [1.4209, 1.7721, 1.8824, 1.5063, 1.6478],
        [1.8896, 1.3686, 1.0754, 1.1028, 1.7788]])

# Define our 
1. model 
2. optimizer 
3. loss function (to optimize)

In [114]:
multi_layer_perceptron = MultiLayerPerceptron(5,3)

# optimizer 
adam = optim.Adam(multi_layer_perceptron.parameters(), lr=1e-1)
print(type(adam))

# loss function 
loss_function = nn.BCELoss()

# calculate how our model is doing now 
y_pred = multi_layer_perceptron(x)
loss_function(y_pred, y).item()

<class 'torch.optim.adam.Adam'>


0.7805761098861694

Let's see if we can have our model achieve a smaller loss. Now that we have everything we need, we can setup our training loop. 

In [117]:
# set number of epoch which determines the number of training iterations. 
n_epoch = 10 

for epoch in range(n_epoch):
    #set gradients to zero 
    adam.zero_grad()

    # get model predictions 
    y_pred = multi_layer_perceptron(x)


    # calculate the loss 
    loss = loss_function(y_pred, y)

    # print stats
    print(f"Epoch {epoch}: training loss: {loss}")

    #computer the gradients 
    loss.backward() 

    # take a step to optimize the weights 
    adam.step()

    


Epoch 0: training loss: 0.7805761098861694
Epoch 1: training loss: 0.465933233499527
Epoch 2: training loss: 0.21142368018627167
Epoch 3: training loss: 0.06858699023723602
Epoch 4: training loss: 0.018102366477251053
Epoch 5: training loss: 0.004522338043898344
Epoch 6: training loss: 0.0011653534602373838
Epoch 7: training loss: 0.00032139578252099454
Epoch 8: training loss: 9.614464215701446e-05
Epoch 9: training loss: 3.128984462819062e-05


In [118]:
list(multi_layer_perceptron.parameters())

[Parameter containing:
 tensor([[ 0.5215,  1.0621,  0.5877,  0.9016,  0.3710],
         [ 0.8491,  0.6657,  0.3032,  1.1651,  1.0538],
         [-0.2273, -0.2718,  0.0138, -0.5835, -0.7417]], requires_grad=True),
 Parameter containing:
 tensor([ 0.6412,  0.5603, -0.8180], requires_grad=True),
 Parameter containing:
 tensor([[ 1.2475,  0.7545,  0.3486],
         [ 1.2015,  0.5252, -0.0629],
         [ 0.7851,  1.0803,  0.8389],
         [ 1.2163,  0.9155, -0.0741],
         [ 0.7432,  1.2263,  0.1078]], requires_grad=True),
 Parameter containing:
 tensor([0.5621, 1.1223, 0.3672, 0.1077, 0.3128], requires_grad=True)]

we can see that our loss is decreasing. Let's check the predictions of our model now and see if they are close to our original y of all ones.

In [120]:
y_pred = multi_layer_perceptron(x)
y_pred

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [0.9999, 0.9999, 0.9999, 1.0000, 0.9999],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000]], grad_fn=<SigmoidBackward0>)

Looks our model almost perfectly learned to filter out the noise from the `x` that we passed in!

# NLP - Word Window Classification 

In this section, our goal will be to train a model that will find the words in a sentence corresponding to a `LOCATION`, which will be always of span `1`. (Meaning tha t`San Fransisco` won't be recognized a s a `LOCATION`). our task is called `Word Window Classification` for a reason. Instead of letting our model to only take a look at one word in each forward pass, we would like tot be able to consider the context of the word in question. That i, for each word, we want our model to be aware of the surrounding words. Let's dive in! 

In [121]:
# Data 

# Our raw corpus. 
corpus = [
    "We always come to Paris", 
    "The processor is from Australia", 
    "I live in Stanford",
    "He comes from Taiwan",
    "The capital of Turkey is Ankara",
]

# Preprocessing

To make it easier for our models to learn, we usually apply a few preprocessing steps to our data. This is especially important when dealing with text data. Here are some examples of text preprocessing: 

* Tokenization: turning sentence in token representation 
* Lowercasing: convert all letters to lowercase 
* Noise Removal: removing special characters (such as punctuations)
* Stop words removal: removing commonly used words 

Which preprocessing steps are necessary is determind by the task at hand. For example, although it is useful to remove special characers in some tasks, for other they may be important( for example, if we are dealing with ultiple langugaes). For our task, we will lowercase our words and tokenize. 

In [123]:
def preprocess_sentence(sentence):
    return sentence.lower().split()

# create our training set 
train_sentences = [preprocess_sentence(sentence) for sentence in corpus]
train_sentences

[['we', 'always', 'come', 'to', 'paris'],
 ['the', 'processor', 'is', 'from', 'australia'],
 ['i', 'live', 'in', 'stanford'],
 ['he', 'comes', 'from', 'taiwan'],
 ['the', 'capital', 'of', 'turkey', 'is', 'ankara']]

for each training example we have, we should also have a corresponding label. Recall that the goal of our model was to determine which words correspond to a `LOCATION`. That is, we want our model to output `0` for all the words that are not `LOCATION` and `1` for the ones that are `LOCATION`s. But we need to create a data for our output 

In [125]:
# set of lcoations tha tappear in our corpus 
locations = set(["australia", "ankara", "paris", "stanford", "taiwan", "turkey"])

# our train labels 
train_labels = [[1 if word in locations else 0 for word in sentence] for sentence in train_sentences]
train_labels

[[0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1]]

# Converting Words to Embeddings 
